### Links zu den Daten
[Strecke Link](https://geovdbn.deutschebahn.com/pgv-map/geoserver.action?LAYERS=ISR%3AISR_V_GEO_TEN_KLASSIFIZIERUNG&TRANSPARENT=TRUE&FORMAT=kml&VERSION=1.1.1&TILED=false&USERDEFINEDSLD=&SERVICE=WMS&REQUEST=GetMap&VIEWPARAMS=ZEITSCHEIBE%3AUNDEFINED%3BLANG%3ADE%3BALG_DBNETZ_STRECKE%3Aalle%20Strecken%3BJAHR%3A2020&SRS=EPSG%3A31467&BBOX=3250000,5200000,3950000,6100000&WIDTH=700&HEIGHT=900)


[Bahnhöfe Link](https://geovdbn.deutschebahn.com/pgv-map/geoserver.action?LAYERS=ISR%3AISR_V_GEO_BETRIEBSSTELLEN_PUNKT&TRANSPARENT=TRUE&FORMAT=kml&VERSION=1.1.1&TILED=false&USERDEFINEDSLD=&SERVICE=WMS&REQUEST=GetMap&VIEWPARAMS=ZEITSCHEIBE%3AUNDEFINED%3BLANG%3ADE%3BALG_DBNETZ_STRECKE%3Aalle%20Strecken%3BJAHR%3A2020&SRS=EPSG%3A31467&BBOX=3250000,5200000,3950000,6100000&WIDTH=700&HEIGHT=900)

In [2]:
import os, sys
sys.path.append("../")
sys.path.append('../rtd_crawler')
import fiona
import geopandas as gpd
from geopandas.plotting import plot_linestring_collection
import matplotlib.pyplot as plt
import shapely
from shapely.geometry import GeometryCollection, MultiPoint, Point
from shapely.ops import nearest_points, linemerge
import networkx as nx
import random
import lxml.etree as etree
import re
import pandas as pd
from tqdm.auto import tqdm
from tqdm import tqdm_notebook as tqdm_bar
from helpers import StationPhillip, BetriebsstellenBill, NoLocationError
import geopy.distance

# enable KML support
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

parser = etree.XMLParser(encoding='utf-8', collect_ids=False)
stations = StationPhillip(notebook=True)
betriebstellen = BetriebsstellenBill(notebook=True)


tqdm.pandas(desc='progress')

def parse_atr(atr):
    atr_name = re.compile(r"<span class=\"atr-name\">(.*?)<\/span>")
    atr_value = re.compile(r"<span class=\"atr-value\">(.*?)<\/span>")

    atr_list = []
    for i in range(len(atr)):
        attributes = atr.iat[i]
        names = [match[1] for match in atr_name.finditer(attributes)]
        values = [match[1] for match in atr_value.finditer(attributes)]
        atr_list.append(pd.Series(values, names))
        
    return pd.concat(atr_list, axis=1).T

def tranform_geo(strecke):
    """
    transform 
    ```python
        GeometryCollection[
            GeometryCollection[
                ...
            ]
        ]
    ```
    to 
    ```python
        Linestring[
            ...
        ]
    ```
    """
    for i in range(len(strecke)):
        new_geo =  []
        for g in strecke.iat[i, 2]:
            if type(g) == shapely.geometry.collection.GeometryCollection:
                for u in g:
                    new_geo.append(u)
            elif type(g) != shapely.geometry.point.Point:
                new_geo.append(g)
        strecke.iat[i, 2] = linemerge(GeometryCollection(new_geo))
    return strecke

def get_map_positions():
    map_positions = {}
    for i in range(len(strecke)):
        if type(strecke.iat[i, 2]) == shapely.geometry.multilinestring.MultiLineString:
            pointroute = MultiPoint(strecke.iat[i, 2][0].coords)
        else:
            try:
                pointroute = MultiPoint(strecke.iat[i, 2].coords)
            except NotImplementedError:
                continue
        map_positions[strecke_atr.loc[i, 'source']] = pointroute[0]
        map_positions[strecke_atr.loc[i, 'target']] = pointroute[-1]
    map_positions = {key: Point(value.x, value.y) for key, value in map_positions.items()}
    return map_positions

def get_strecke_atr():
    strecke_atr = parse_atr(strecke.iloc[:, 1])
    strecke_atr['source'] = ''
    strecke_atr['target'] = ''
    for i in range(len(strecke_atr)):
        nodes = strecke_atr.at[i, 'ISR_STRECKE_VON_BIS'].split(' - ')
        strecke_atr.at[i, 'source'] = nodes[0]
        strecke_atr.at[i, 'target'] = nodes[1]
    return strecke_atr

def calculate_nearest(row, destination, val, unary_union, col="geometry"):
    try:
        nearest_geom = nearest_points(row[col], dest_unary)
        match_geom = destination.loc[destination.geometry == nearest_geom[1]]
        match_value = match_geom[val].to_numpy()[0]
        return match_value
    except:
        return None

Using local station buffer
Using local betriebstellen buffer


In [8]:
def name_replace(name):
    return name.lower().replace(' ', '').replace('(', '').replace(')', '').replace('.', '')

In [10]:
name_replace('Tübingen (Hbf).')

'tübingenhbf'

In [3]:
strecke = gpd.read_file('../data/ISR-ISR_V_GEO_TEN_KLASSIFIZIERUNG.kml')
bahnhöfe = gpd.read_file('../data/ISR-ISR_V_GEO_BETRIEBSSTELLEN_PUNKT.kml')

In [4]:
strecke = tranform_geo(strecke)

In [5]:
strecke_atr = get_strecke_atr()

In [6]:
map_positions = get_map_positions()

In [7]:
strecke_graph = nx.from_pandas_edgelist(strecke_atr, source='source', target='target')

In [7]:
sta_pos = []
for i in range(len(stations)):
    sta_pos.append(Point(stations.station_df.at[i, 'lon'], stations.station_df.at[i, 'lat']))

In [8]:
sta_pos = MultiPoint(sta_pos)

In [9]:
map_positions_df = pd.DataFrame({'bhf':list(map_positions.keys()), 'location':list(map_positions.values())})
map_positions_gdf = gpd.GeoDataFrame(map_positions_df, geometry=map_positions_df['location'])

In [10]:
stations_gdf = stations.get_geopandas()

In [11]:
map_positions_points = MultiPoint(list(map_positions.values()))

In [13]:
dest_unary = map_positions_gdf["geometry"].unary_union

In [23]:
stations_gdf['nearest_geom'] = stations_gdf.progress_apply(calculate_nearest, destination=map_positions_gdf, unary_union=dest_unary, val='geometry', axis=1)

my bar!: 100%|██████████| 7603/7603 [24:20<00:00,  5.21it/s]


In [27]:
stations_gdf['nearest_node'] = stations_gdf.progress_apply(calculate_nearest, destination=map_positions_gdf, unary_union=dest_unary, val='bhf', axis=1)

my bar!: 100%|██████████| 7603/7603 [24:36<00:00,  5.15it/s]


In [31]:
stations_gdf.to_csv('station_matches.csv')

In [ ]:
nearest = []
for i in tqdm(range(len(sta_pos))):
    try:
        nearest.append(nearest_points(sta_pos[i], map_positions_points)[1])
    except ValueError:
        nearest.append(None)

In [ ]:
distance = []
for i in tqdm(range(len(nearest))):
    try:
        distance.append(geopy.distance.distance(sta_pos[i].coords, nearest[i].coords).km)
    except AttributeError:
        distance.append(None)

In [ ]:
def get_to_far(threshold):
    to_far = 0
    for dist in distance:
        if dist and dist > threshold:
            to_far += 1
    return to_far

In [ ]:
for station_coords in tqdm(nearest):
    print(station_coords)
    list(sta_pos).index(station_coords)

In [ ]:
data = {1 /(threshold ** 2): get_to_far((1 /(threshold ** 2))) for threshold in range(5, 15)}

In [ ]:
plt.plot(list(data.keys()), list(data.values()))

In [ ]:
positions = {}
for station in strecke_graph.nodes():
    try:
        positions[station] = betriebstellen.get_location(name=station)
    except KeyError:
        continue
    except NoLocationError:
        continue

In [ ]:
pos = nx.spring_layout(strecke_graph, pos=map_positions, fixed=map_positions.keys(), k=0.001)

In [ ]:
fig, ax = plt.subplots(figsize=(21*2, 24*2))
ax = nx.draw(strecke_graph, pos=pos, ax=ax)
bahnhöfe.plot(ax=ax)

In [ ]:
number_of_colors = len(strecke)

color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]
strecke['colors'] = color

In [ ]:
ax = strecke.plot(column='colors', figsize=(21*2, 24*2))
bahnhöfe.plot(ax=ax)

In [ ]:
bahnhöfe.plot(figsize=(21*2, 24*2))